In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-i5_ftpsx/unsloth_ba4c7a8dab0142acaedb90e35b5e9a96
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-i5_ftpsx/unsloth_ba4c7a8dab0142acaedb90e35b5e9a96
  Resolved https://github.com/unslothai/unsloth.git to commit 229e2ecc67756f36316dfcbea42396f59eef44e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of trl to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/fiap/news_dataset_chat_data.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/fiap/formatted_news_dataset_chat_data.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        news_start = full_text.find("[|News|]") + len("[|News|]")
        news_end = full_text.find("[|eNews|]")
        summary_start = full_text.find("[|summary|]") + len("[|summary|]")
        summary_end = full_text.find("[|esummary|]")

        instruction = full_text.split('\n')[0]
        input_text = full_text[news_start:news_end].strip()
        response = full_text[summary_start:summary_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    for prompt in data['train']['input']:
        instruction, input_text, response = separate_text(prompt)
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

In [7]:
data = load_dataset("json", data_files=DATA_PATH)

In [8]:

format_dataset_into_model_input(data)


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Dataset salvo em /content/drive/MyDrive/fiap/formatted_news_dataset_chat_data.json
==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.9.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [10]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/fiap/formatted_news_dataset_chat_data.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10467 [00:00<?, ? examples/s]

In [11]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/10467 [00:00<?, ? examples/s]

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,467 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rodrims2003 (rodrims2003-letsdev-tecnologia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.102700
2,2.087900
3,2.038900
4,1.977500
5,2.133900
6,2.252600
7,2.126600
8,2.014300
9,2.050000
10,2.009700


In [13]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS NEWS",
        "If home is where the heart is, a new survey suggests that most people aren't sure exactly where they live. More than half of people cannot pinpoint the exact location of the human heart on a diagram, and nearly 70 percent can't correctly identify the shape of the lungs, according to the survey. This lack of knowledge isn't just embarrassing -- it could lead to a poorer quality of health care, some experts say. In the study, published in the journal BMC Family Practice, a research team surveyed 722 Britons -- 589 hospital outpatients and 133 people in the general population. They gave the volunteers four diagrams of human figures and asked them to choose the one that showed the correct size and location of a specific organ. (For example, the heart diagrams showed various size organs on the far left side of the chest, directly in the center, anchored on the center/left chest, and on the right side of the chest.) Overall, people knew less basic anatomy than the researchers expected -- even those patients being treated for a specific condition involving that organ. Participants generally answered half the questions correctly, including 46.5 percent who knew which drawing represented their heart. In all, 31.4 percent correctly identified the lungs, 38.4 percent the stomach, 41.8 percent the thyroid, and 42.5 percent the kidneys. The intestines and bladder were the most easily identified, with 85.9 percent and 80.7 percent, respectively, answering the question correctly. Health.com: Are you cholesterol smart? Take this quiz There was little to no improvement compared with a similar study conducted in 1970, says lead author John Weinman, Ph.D., of King's College London. In that study, subjects correctly identified eight major body parts about half of the time. (The researchers used the same body parts from the 1970 study and added three more: the pancreas, gallbladder, and ovaries.) Given the accessibility of the Internet and the prominence of health stories in the news media today, Weinman's team expected that people would now know more about their body. Weinman says he wouldn't be surprised if a study based in the United States produced similar results -- or worse. \"I imagine they would be similar, but there could well be regional variation, depending on which part of the U.S. the participants were from,\" he says. \"Actually, I asked one of my colleagues, who is from the U.S., and she felt that Americans might be worse because, to quote her, 'Very many Americans don't even know where New Jersey is, so how would they know where their pancreas is?'\" Health.com: Eat Smarter in your 30s, 40s, and 50s That may sound harsh, but time and again, U.S. studies have shown that doctors overestimate how much their patients understand about their conditions and treatment. Adam Kelly, Ph.D., an assistant professor of medicine at the Baylor College of Medicine, in Houston,  Texas, conducted a 2007 study, that showed that doctors overestimate patient literacy and that a lack of patient knowledge leads to poorer care. Kelly believes the problem could be \"even more profound\" in the United States, although a similar study has not been conducted in America. Still, anatomy may not be the best measure of health literacy, says Sandeep Jauhar, M.D., the director of the heart failure program at the Long Island Jewish Medical Center, and the author of \"Intern: A Doctor's Initiation.\" Health.com: Computer games that boost your memory \"They would like us to draw the conclusion that because the patients can't identify these organs anatomically that that is an indication of low health-care literacy -- and that may or may not be true,\" he says. \"I work with heart failure patients, and whether they can identify where their heart is is not so important to me as long as they know which medicines to take and when.\" Many patients with heart failure, unfortunately, don't know which medicines to take, can't identify their symptoms, and don't follow up with their doctors, Jauhar says.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSUMMARIZE THIS NEWS\n\n### Input:\nIf home is where the heart is, a new survey suggests that most people aren\'t sure exactly where they live. More than half of people cannot pinpoint the exact location of the human heart on a diagram, and nearly 70 percent can\'t correctly identify the shape of the lungs, according to the survey. This lack of knowledge isn\'t just embarrassing -- it could lead to a poorer quality of health care, some experts say. In the study, published in the journal BMC Family Practice, a research team surveyed 722 Britons -- 589 hospital outpatients and 133 people in the general population. They gave the volunteers four diagrams of human figures and asked them to choose the one that showed the correct size and location of a specific organ. (For example, the heart diagram

In [14]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS NEWS",
        "If home is where the heart is, a new survey suggests that most people aren't sure exactly where they live. More than half of people cannot pinpoint the exact location of the human heart on a diagram, and nearly 70 percent can't correctly identify the shape of the lungs, according to the survey. This lack of knowledge isn't just embarrassing -- it could lead to a poorer quality of health care, some experts say. In the study, published in the journal BMC Family Practice, a research team surveyed 722 Britons -- 589 hospital outpatients and 133 people in the general population. They gave the volunteers four diagrams of human figures and asked them to choose the one that showed the correct size and location of a specific organ. (For example, the heart diagrams showed various size organs on the far left side of the chest, directly in the center, anchored on the center/left chest, and on the right side of the chest.) Overall, people knew less basic anatomy than the researchers expected -- even those patients being treated for a specific condition involving that organ. Participants generally answered half the questions correctly, including 46.5 percent who knew which drawing represented their heart. In all, 31.4 percent correctly identified the lungs, 38.4 percent the stomach, 41.8 percent the thyroid, and 42.5 percent the kidneys. The intestines and bladder were the most easily identified, with 85.9 percent and 80.7 percent, respectively, answering the question correctly. Health.com: Are you cholesterol smart? Take this quiz There was little to no improvement compared with a similar study conducted in 1970, says lead author John Weinman, Ph.D., of King's College London. In that study, subjects correctly identified eight major body parts about half of the time. (The researchers used the same body parts from the 1970 study and added three more: the pancreas, gallbladder, and ovaries.) Given the accessibility of the Internet and the prominence of health stories in the news media today, Weinman's team expected that people would now know more about their body. Weinman says he wouldn't be surprised if a study based in the United States produced similar results -- or worse. \"I imagine they would be similar, but there could well be regional variation, depending on which part of the U.S. the participants were from,\" he says. \"Actually, I asked one of my colleagues, who is from the U.S., and she felt that Americans might be worse because, to quote her, 'Very many Americans don't even know where New Jersey is, so how would they know where their pancreas is?'\" Health.com: Eat Smarter in your 30s, 40s, and 50s That may sound harsh, but time and again, U.S. studies have shown that doctors overestimate how much their patients understand about their conditions and treatment. Adam Kelly, Ph.D., an assistant professor of medicine at the Baylor College of Medicine, in Houston,  Texas, conducted a 2007 study, that showed that doctors overestimate patient literacy and that a lack of patient knowledge leads to poorer care. Kelly believes the problem could be \"even more profound\" in the United States, although a similar study has not been conducted in America. Still, anatomy may not be the best measure of health literacy, says Sandeep Jauhar, M.D., the director of the heart failure program at the Long Island Jewish Medical Center, and the author of \"Intern: A Doctor's Initiation.\" Health.com: Computer games that boost your memory \"They would like us to draw the conclusion that because the patients can't identify these organs anatomically that that is an indication of low health-care literacy -- and that may or may not be true,\" he says. \"I work with heart failure patients, and whether they can identify where their heart is is not so important to me as long as they know which medicines to take and when.\" Many patients with heart failure, unfortunately, don't know which medicines to take, can't identify their symptoms, and don't follow up with their doctors, Jauhar says.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS NEWS

### Input:
If home is where the heart is, a new survey suggests that most people aren't sure exactly where they live. More than half of people cannot pinpoint the exact location of the human heart on a diagram, and nearly 70 percent can't correctly identify the shape of the lungs, according to the survey. This lack of knowledge isn't just embarrassing -- it could lead to a poorer quality of health care, some experts say. In the study, published in the journal BMC Family Practice, a research team surveyed 722 Britons -- 589 hospital outpatients and 133 people in the general population. They gave the volunteers four diagrams of human figures and asked them to choose the one that showed the correct size and location of a specific organ. (For example, the heart diagrams showed va

In [15]:
model.save_pretrained("/content/drive/MyDrive/fiap/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/fiap/lora_model")


('/content/drive/MyDrive/fiap/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/fiap/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/fiap/lora_model/tokenizer.json')

In [16]:
import json
news_test = []

with open('/content/drive/MyDrive/fiap/test_news/news_contents.json', 'r') as news_file:
  news_test = json.loads(news_file.read())['news_content']

In [17]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/fiap/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)



==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [20]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
alpaca_prompt.format(
        "SUMMARIZE THIS NEWS",
        news_test[3],
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS NEWS

### Input:
A huge and powerful typhoon is headed toward southern China afterlashing the Philippineswith destructive winds and torrential rain, putting the region’s megacities on high alert with cancelled flights and disruptions to schools and businesses. Tens of millions of people could be impacted by the storm, which is expected to pass south of the major cities of Hong Kong, Macau, Shenzhen and Guangzhou, before making landfall again in mainland China’s Guangdong Province. Typhoon Ragasahit the northern Philippines as the strongest storm on earth so far this year, after generating sustained winds of over165 mph (267 kph), the equivalent of a Category 5 hurricane. The sheer size of the system churning its way through the South China Sea was captured from space by Japanese as